<a href="https://colab.research.google.com/github/JAaron93/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/Train_a_GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: February 14th, 2021*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [2]:
!nvidia-smi

Sun Aug 15 21:08:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [3]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 635Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 3.28Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 804Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:14, 35.1Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 710Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.01Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.96Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [4]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [5]:
import pandas as pd
training = "training.csv"
train = pd.read_csv(training)

In [6]:
train.head()

,Unnamed: 0,id,tweets,labels
0,0,6,A New Mexico State Police officer killed two p...,5
1,1,7,I found some police badge stickers at work and...,0
2,2,8,"According to media reports, police said they a...",1
3,3,9,The male who has the knife is now running nort...,0
4,4,10,Changing the assignments of already existing c...,0


In [33]:
train.drop(['Unnamed: 0'], axis=1, inplace=True)

rank2 = pd.DataFrame(train[train['labels'] == 2])
rank2 = rank2['tweets']
rank3 = pd.DataFrame(train[train['labels'] == 3])
rank3 = rank3['tweets']
rank4 = pd.DataFrame(train[train['labels'] == 4])
rank4 = rank4['tweets']

In [35]:
rank2

119     UPDATE: Vacaville Police K9 Punched in the Fac...
131     New York police department,made in Israel!! Wa...
175     Bank holiday brawl between police and pub punt...
187     5 Bangladeshi nationals arrested in Bengaluru ...
200     4 volunteers with Black Protest Legal Support ...
                              ...                        
5999    Reporter Josie Huang was covering the shooting...
6010    Footage and reports show approximately 30 poli...
6019    Footage from 3 tiktok videos shows numerous of...
6024    Police seemingly single out a person that is b...
6027    I fully condemn the police brutality at the fr...
Name: tweets, Length: 177, dtype: object

In [36]:
rank2.to_csv('rank2tweets.txt', sep='\t')
rank3.to_csv('rank3tweets.txt', sep='\t')
rank4.to_csv('rank4tweets.txt', sep='\t')

In [5]:
file_name = "rank2tweets.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [6]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [7]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500,
              overwrite=True
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  8.10it/s]

dataset has 12389 tokens
Training...


[10 | 28.40] loss=2.50 avg=2.50
[20 | 50.03] loss=1.71 avg=2.11
[30 | 71.89] loss=1.18 avg=1.80
[40 | 93.98] loss=0.59 avg=1.49
[50 | 116.27] loss=0.24 avg=1.23
[60 | 138.74] loss=0.12 avg=1.04
[70 | 161.34] loss=0.07 avg=0.90
[80 | 184.08] loss=0.06 avg=0.79
[90 | 206.98] loss=0.10 avg=0.71
[100 | 229.89] loss=0.05 avg=0.64
[110 | 252.71] loss=0.04 avg=0.58
[120 | 275.58] loss=0.03 avg=0.54
[130 | 298.46] loss=0.04 avg=0.50
[140 | 321.35] loss=0.04 avg=0.46
[150 | 344.26] loss=0.04 avg=0.43
[160 | 367.17] loss=0.05 avg=0.40
[170 | 390.07] loss=0.04 avg=0.38
[180 | 412.97] loss=0.03 avg=0.36
[190 | 435.91] loss=0.03 avg=0.34
[200 | 458.81] loss=0.03 avg=0.32
======== SAMPLE 1 ========
 fled. A reporter attempting to film is harassed and told to move faster by a second officer. The second officer shoves the journalist. The journalist then staggers away on the sidewalk before a second officer stops him and asks him why he's being arrested. The officer again shoves the journalist. The rep

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [8]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [9]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name='run1')

Romeo was a gentleman,
A gentleman of gentle complexion,
A gentleman in gentle discourse,
A gentleman in gentle words,
A gentleman in gentle discourse,
To smile at gentle Polixenes,
And smile again at gentle Menenius.

MENENIUS:
A gentle discourse,
And a very mild one indeed,
For such a subject as ours,
To smooth the wrinkles of the brow.

First Senator:
Our subjects,
We are pleased to send you on your first pleasure,
After your first pleasure we have well proceeded.

VOLUMNIA:
I think, sir, about the wine, sir, and the rest,
Would you advise each of them in their turn to bring
The wine to them?

CORIOLANUS:
Yea, sir; I'll be in patience;
And let you bequeath the wine to your wine,
If you so desire.

MENENIUS:
Would you prescribe any thing?

CORIOLANUS:
I will in person,
Wherein the physician is well acquainted.

MENENIUS:
Well, no more: you have arrived at the right
Till you have acquainted me with Valentinus.

CORIOLANUS:
The very first pleasure I had in your daughter,
To see her at 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="LORD",
              nsamples=5,
              batch_size=5
              )

LORD WILLOUGHBY:
And, Lord Marshal, to the Tower.

KING RICHARD III:
As near the time when we were born,
Let not our graces enter into this story,
For God's sake, the whole truth shall stand:
The selfsame night God gave the sun,
Let angels, and stars enter, and the four cardinal directions,
Be satisfied, that the power thereof is determined:
For we are ere now the prime of corn,
Or of harvest, harvest being done,
Caught in the vice of night, and fall asleep.
O Clifford, how we lay to bed! Clifford, I am loath
To sleep under this false bed of roses:
I think, to mellow out this bed, I may by a jest
Be naked and naked and naked! If night be pall'd,
All self-portents must fall asleep;
Unless, God forbid, thou art not in bed.

QUEEN ELIZABETH:
I am, and therefore, as thou suspect'st, O.K.
Where there is no strength in the dark,
Never will I rest, till thou bring me
LORD WILLOUGHBY:
The morn to pray is vain;
Nor doth it offend the holy church,
Nor any holy man. Come, go with me;
I will not l

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 354Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 131Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 279Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:23, 131Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 380Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 226Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 199Mit/s]                                                       


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

W0828 18:37:58.571830 139905369159552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Loading pretrained model models/774M/model.ckpt


In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

The secret of life is that it's really easy to make it complicated," said Bill Nye, the host of the popular science show "Bill Nye the Science Guy." "And this is one of the reasons why we all need to be smarter about science, because we can't keep up with the amazing things that are going on all the time."

While Nye is correct that "everything that's going on all the time" is making the world a better place, he misses the point. This is not
The secret of life is in the rhythm of the universe. It's not a mystery. It's not a mystery to me. It's the nature of the universe. It's the beauty of the universe. It's the way the universe works. It's the way the universe is. It's the way the universe is going to work. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way
The secret of life is in the universe.


-

The Red Devil

It's the end of the world as we know it, and the only thing that can save us is a band of 

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.